In [ ]:
!pip install huggingface_hub boto3 -q

!wget -qO- https://releases.hashicorp.com/terraform/1.8.0/terraform_1.8.0_linux_amd64.zip > terraform.zip
!unzip -q terraform.zip
!mv terraform /usr/local/bin/
!terraform -version


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 5.6 MB/s eta 0:00:00
Terraform v1.8.0
on linux_amd64

Your version of Terraform is out of date! The latest version
is 1.12.2. You can update by downloading from https://www.terraform.io/downloads.html


In [ ]:
import os
import subprocess
import boto3
import uuid
from huggingface_hub import InferenceClient, login

# 🔐 Add your credentials
# AWS_ACCESS_KEY_ID = "your-access-key"
# AWS_SECRET_ACCESS_KEY = "your-secret-key"



AWS_REGION = "us-east-1"



# Hugging Face login
login(token=HF_TOKEN)

# Hugging Face client2
client = InferenceClient(model="mistralai/Mistral-7B-Instruct-v0.", token=HF_TOKEN)

# Set AWS credentials for Terraform
os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY

# Boto3 EC2 client
ec2 = boto3.client("ec2", region_name=AWS_REGION,
                   aws_access_key_id=AWS_ACCESS_KEY_ID,
                   aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

# Working directory
os.makedirs("/content/terraform", exist_ok=True)
%cd /content/terraform


/content/terraform


In [ ]:
def generate_terraform_code(prompt):
    system_instruction = (
        "You are a DevOps engineer. Generate only valid Terraform configuration code "
        "based on this instruction. Respond with Terraform HCL only — no extra text and ensure the AMI is right."
    )
    response = client.chat_completion(
        messages=[
            {"role": "system", "content": system_instruction},
            {"role": "user", "content": prompt}
        ],
        max_tokens=600,
        temperature=0.3
    )

    code = response.choices[0].message["content"]

    # Remove any extra lines like comments or markdown formatting
    lines = code.strip().splitlines()
    cleaned_lines = [line for line in lines if not line.strip().startswith("Replace") and not line.strip().startswith("```")]
    return "\n".join(cleaned_lines).strip()


In [ ]:
def get_default_subnet_id():
    subnets = ec2.describe_subnets()
    subnet = subnets["Subnets"][0]
    return subnet["SubnetId"], subnet["VpcId"]

def create_key_pair():
    key_name = f"key-{uuid.uuid4().hex[:8]}"
    key_pair = ec2.create_key_pair(KeyName=key_name)
    key_file = f"{key_name}.pem"
    with open(key_file, "w") as f:
        f.write(key_pair["KeyMaterial"])
    os.chmod(key_file, 0o400)
    return key_name, key_file

def get_or_create_security_group(vpc_id):
    sg_name = "terraform-agent-sg"

    # Check if SG already exists
    existing_sgs = ec2.describe_security_groups(
        Filters=[
            {"Name": "group-name", "Values": [sg_name]},
            {"Name": "vpc-id", "Values": [vpc_id]}
        ]
    )

    if existing_sgs["SecurityGroups"]:
        return existing_sgs["SecurityGroups"][0]["GroupId"]

    # Create a new SG
    response = ec2.create_security_group(
        GroupName=sg_name,
        Description="SG for Terraform agent",
        VpcId=vpc_id
    )
    sg_id = response["GroupId"]

    # Allow SSH ingress (port 22)
    ec2.authorize_security_group_ingress(
        GroupId=sg_id,
        IpPermissions=[
            {
                "IpProtocol": "tcp",
                "FromPort": 22,
                "ToPort": 22,
                "IpRanges": [{"CidrIp": "0.0.0.0/0"}]
            }
        ]
    )
    return sg_id

def replace_all_placeholders(tf_code, key_name, subnet_id, sg_id):
    return (
        tf_code
        .replace("your_key_name", key_name)
        .replace("your-key-pair-name", key_name)
        .replace("your_subnet_id", subnet_id)
        .replace("your-subnet-id", subnet_id)
        .replace("your-security-group-id", sg_id)
    )


In [ ]:
def validate_terraform():
    print("📋 Running terraform validate...")
    result = subprocess.run(["terraform", "validate"], capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print("❌ Terraform validation failed:")
        print(result.stderr)
        raise subprocess.CalledProcessError(result.returncode, result.args)





def run_terraform():
    print("🔧 terraform init...")
    init = subprocess.run(["terraform", "init"], capture_output=True, text=True)
    print(init.stdout)
    if init.returncode != 0:
        print("❌ Terraform init failed:")
        print(init.stderr)
        return

    validate_terraform()

    print("🚀 terraform apply...")
    apply = subprocess.run(["terraform", "apply", "-auto-approve"], capture_output=True, text=True)
    print(apply.stdout)
    if apply.returncode != 0:
        print("❌ Terraform apply failed:")
        print(apply.stderr)
        raise subprocess.CalledProcessError(apply.returncode, apply.args, output=apply.stdout, stderr=apply.stderr)


def destroy_terraform():
    subprocess.run(["terraform", "destroy", "-auto-approve"], check=True)

def get_instance_ips():
    instances = ec2.describe_instances(Filters=[
        {"Name": "instance-state-name", "Values": ["running"]}
    ])
    ips = []
    for res in instances["Reservations"]:
        for inst in res["Instances"]:
            ip = inst.get("PublicIpAddress")
            if ip:
                ips.append(ip)
    return ips


In [ ]:
def save_code_to_file(code):
    with open("main.tf", "w") as f:
        f.write(code)

In [ ]:
def run_agent():
    while True:
        prompt = input("\n📥 What would you like to provision? Type 'destroy' or 'exit':\n> ")
        if prompt.lower() == "exit":
            break
        elif prompt.lower() == "destroy":
            destroy_terraform()
            print("🗑️ Infrastructure destroyed.")
        else:
            print("🧠 Generating Terraform configuration...")
            raw_code = generate_terraform_code(prompt)

            print("🔍 Creating AWS key pair and resolving network resources...")
            key_name, key_file = create_key_pair()
            subnet_id, vpc_id = get_default_subnet_id()
            sg_id = get_or_create_security_group(vpc_id)

            print(f"🔐 Key: {key_name} | Subnet: {subnet_id} | SG: {sg_id}")
            final_code = replace_all_placeholders(raw_code, key_name, subnet_id, sg_id)
            save_code_to_file(final_code)

            print("\nGenerated Terraform Code:\n")
            print(final_code)

            print("\n--- Final Terraform Configuration (main.tf) ---\n")
            with open("main.tf") as f:
             print(f.read())


            print("⚙️  Running Terraform...")
            run_terraform()

            ips = get_instance_ips()
            print("✅ EC2 Instances created. Public IPs:")
            for ip in ips:
                print(f"🔗 {ip}")

            print(f"\n🔑 Your private key is saved as: {key_file} (download before ending session!)")


In [ ]:
run_agent()


📥 What would you like to provision? Type 'destroy' or 'exit':
> Create an s3 buket and call it july1420251234 DO NOT ADD ANY COMMENT IN THE FILE and DO nOT RECREATE EC2 and THE DO NOT ADD THE VERSION.
🧠 Generating Terraform configuration...
🔍 Creating AWS key pair and resolving network resources...
🔐 Key: key-ce803e0e | Subnet: subnet-0bc91d699e42a021f | SG: sg-00cd5b7dbb4359ba6

Generated Terraform Code:

provider "aws" {
 version = "4.36.0"
 region = "us-west-2"
}

resource "aws_s3_bucket" "example" {
 name = "july1420251234"
 acl = "private"
}

--- Final Terraform Configuration (main.tf) ---

provider "aws" {
 version = "4.36.0"
 region = "us-west-2"
}

resource "aws_s3_bucket" "example" {
 name = "july1420251234"
 acl = "private"
}
⚙️  Running Terraform...
🔧 terraform init...

Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/aws from the dependency lock file
╷
│ Warning: Version constraints inside provider configuration blocks a

In [ ]:
#  Create four t2.micro EC2 instances with Ubuntu in us-east-1, and ensure the code is valid for terraform to run, DO NOT INCLUDE SECUTITY GROUP AND SUBNET AND DO NOT ADD ANY COMMENT IN THE FILE and no version. use this ami ami-020cba7c55df1f615

In [ ]:
Create an s3 buket and call it july1420251234 DO NOT ADD ANY COMMENT IN THE FILE and DO nOT RECREATE EC2 and THE DO NOT ADD THE VERSION.